In [5]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
from scipy.stats import spearmanr
import scanpy.external as sce
from itertools import product

In [2]:
path_cwd=os.getcwd()
path_train_data=path_cwd + '/ML4G_Project_2_Data/train_data'
path_test_data=path_cwd + '/ML4G_Project_2_Data/test_data'

In [3]:
#train_data
bulk_train=pd.read_csv(path_train_data +'/pancreas_bulk_train.csv',index_col=0)
metadata_train=pd.read_csv(path_train_data+ '/pancreas_sc_metadata_train.csv')
sc_train=pd.read_csv(path_train_data +'/pancreas_sc_train.csv', index_col=0)

#test_data
metadata_test=pd.read_csv(path_test_data+ '/pancreas_sc_metadata_test_wocelltype.csv')
sc_test=pd.read_csv(path_test_data +'/pancreas_sc_test.csv', index_col=0)

In [4]:
cells_1=[column for column in sc_train.columns if 'patient1' in column ]
cells_2=[column for column in sc_train.columns if 'patient2' in column ]
cells_3=[column for column in sc_train.columns if 'patient3' in column ]
cells_4=[column for column in sc_train.columns if 'patient4' in column ]

# we transpose the dataframe since we want a cells x genes dataframe for applying MAGIC
sc1_train=sc_train[cells_1].T
sc2_train=sc_train[cells_2].T
sc3_train=sc_train[cells_3].T
sc4_train=sc_train[cells_4].T


genes1=list(sc1_train.loc[:, (sc1_train != sc1_train.iloc[0]).any()].columns)
genes2=list(sc2_train.loc[:, (sc2_train != sc2_train.iloc[0]).any()].columns)
genes3=list(sc3_train.loc[:, (sc3_train != sc3_train.iloc[0]).any()].columns)
genes4=list(sc4_train.loc[:, (sc4_train != sc4_train.iloc[0]).any()].columns)


genes=[genes1,genes2,genes3,genes4]
sc1_train.head(3)

Unnamed: 0,SGIP1,AZIN2,CLIC4,AGBL4,NECAP2,SLC45A1,TGFBR3,DBT,RFWD2,C1orf21,...,LOC389831,MGC70870,KIR2DS5,KIR2DL5A,KIR3DS1,KIR2DL5B,KIR2DS2,KIR2DS1,KIR2DL2,KIR2DS3
patient1_A10,0.0,0.0,127.0,0.0,140.0,0.0,0.0,1.0,103.0,267.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
patient1_A12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
patient1_A13,0.0,110.0,0.0,21.0,0.0,0.0,0.0,56.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Pre processing: Normalize per total reads per cell and transform with log
def pp(data):
    data=ad.AnnData(data) # needed to use following functions
    #sc.pp.filter_cells(data,min_counts=1)
    sc.pp.normalize_total(data)
    sc.pp.sqrt(data)
    data=pd.DataFrame(data.X, columns = data.var_names,index=data.obs_names)
    return data

patients=[sc1_train,sc2_train,sc3_train,sc4_train]
patients_pp=[]
#after the loop variables are called data1,data2,data3,data4 or are in the patients_pp list
for i,data in enumerate(patients):
    pp_data=pp(data)
    vars().__setitem__('data' + str(i+1), pp_data )
    patients_pp += [pp_data]

In [116]:
'''results=[]
k_set ,t_set, n_pca_set= [3,5,7,10], list(range(1,5))+ ['auto'] ,[70,90,110]
n=1
for k,t,n_pca in product(k_set,t_set,n_pca_set):
    if n%5==0: print(n)
    result=0
    for i,patient in enumerate(patients_pp):
        imputed_global=patient.copy()
        imputed_local=sce.pp.magic(ad.AnnData(patient.loc[:,genes[i]]),knn=k,t=t,n_pca=n_pca,n_jobs=8,copy=True,verbose=0,solver='approximate')
        imputed_global.loc[:,genes[i]]=imputed_local.X
        pred=imputed_global.mean(axis=0)
        bulk=bulk_train['patient' + str(i+1)]
        result+=spearmanr(pred,bulk).statistic/4
    results+=[result]
    n+=1

baseline=np.mean([spearmanr(patients_pp[i].mean(axis=0),bulk_train['patient' + str(i+1)]).statistic for i in range(4)])
print(f'Correlation before imputation: {baseline}')
print(f'Best Correlation: {max(results)}')

best_iter=np.argmax(results)
i=0
for a,b,c in product(k_set,t_set,n_pca_set):
    if i==best_iter:
        print(a,b,c)
i+=1
'''

"results=[]\nk_set ,t_set, n_pca_set= [3,5,7,10], list(range(1,5))+ ['auto'] ,[70,90,110]\nn=1\nfor k,t,n_pca in product(k_set,t_set,n_pca_set):\n    if n%5==0: print(n)\n    result=0\n    for i,patient in enumerate(patients_pp):\n        imputed_global=patient.copy()\n        imputed_local=sce.pp.magic(ad.AnnData(patient.loc[:,genes[i]]),knn=k,t=t,n_pca=n_pca,n_jobs=8,copy=True,verbose=0,solver='approximate')\n        imputed_global.loc[:,genes[i]]=imputed_local.X\n        pred=imputed_global.mean(axis=0)\n        bulk=bulk_train['patient' + str(i+1)]\n        result+=spearmanr(pred,bulk).statistic/4\n    results+=[result]\n    n+=1\n\nbaseline=np.mean([spearmanr(patients_pp[i].mean(axis=0),bulk_train['patient' + str(i+1)]).statistic for i in range(4)])\nprint(f'Correlation before imputation: {baseline}')\nprint(f'Best Correlation: {max(results)}')\n\nbest_iter=np.argmax(results)\ni=0\nfor a,b,c in product(k_set,t_set,n_pca_set):\n    if i==best_iter:\n        print(a,b,c)\ni+=1\n"

# New Approach: Using the whole dataset

In [120]:
data_complete=pd.concat(patients_pp,axis=0)
genes=list(data_complete.loc[:, (data_complete != data_complete.iloc[0]).any()].columns)
lens=[len(x) for x in patients_pp]
indexes=[np.sum(lens[:i+1]) for i in range(4)]

In [121]:
results=[]
k_set ,t_set, n_pca_set= [60,80,90,100], ['auto'] ,[175,180,185]
#k_set ,t_set, n_pca_set= [70,75], ['auto'] ,[175,180,185]
best_params=0
best_score=0
baseline=np.mean([spearmanr(patients_pp[i].mean(axis=0),bulk_train['patient' + str(i+1)]).statistic for i in range(4)])
for k,t,n_pca in product(k_set,t_set,n_pca_set):
    imputed_global=data_complete.copy()
    imputed_local=sce.pp.magic(ad.AnnData(data_complete.loc[:,genes]),knn=k,t=t,n_pca=n_pca,n_jobs=8,copy=True,verbose=0,solver='approximate',random_state=7)
    imputed_global.loc[:,genes]=imputed_local.X
    result=0
    for i,index in enumerate(indexes):
        if i==0: sc_imputed= imputed_global.iloc[:index,:]
        else: sc_imputed= imputed_global.iloc[indexes[i-1]:index,:]
        bulkified=sc_imputed.mean(axis=0)
        bulk=bulk_train['patient' + str(i+1)]
        result+=spearmanr(bulkified,bulk).statistic/4
    if result >= best_score:
        best_params=[k,t,n_pca]
        best_score=result
    results+=[result]
    print(f'parameters: {k,t,n_pca}')
    print(f'Correlation After imputation: {result}\n--------')


print(f'Correlation before imputation: {baseline}')
print(f'BEST CORRELATION AFTER IMPUTATION: {np.max(results)}')
print(f'best parameters: {best_params}')

parameters: (60, 'auto', 175)
Correlation After imputation: 0.9451095086701955
--------
parameters: (60, 'auto', 180)
Correlation After imputation: 0.9450754418163375
--------
parameters: (60, 'auto', 185)
Correlation After imputation: 0.9451700908929492
--------
parameters: (80, 'auto', 175)
Correlation After imputation: 0.9458508261935643
--------
parameters: (80, 'auto', 180)
Correlation After imputation: 0.9458453546344455
--------
parameters: (80, 'auto', 185)
Correlation After imputation: 0.9459011722561625
--------
parameters: (90, 'auto', 175)
Correlation After imputation: 0.9458156154267131
--------
parameters: (90, 'auto', 180)
Correlation After imputation: 0.9457986153794349
--------
parameters: (90, 'auto', 185)
Correlation After imputation: 0.9458331655369938
--------
parameters: (100, 'auto', 175)
Correlation After imputation: 0.9456221658779189
--------
parameters: (100, 'auto', 180)
Correlation After imputation: 0.9456677206530077
--------
parameters: (100, 'auto', 185)



# New Approach: Differentiating by health status

In [96]:
data_complete_healthy=pd.concat(patients_pp[1:3],axis=0)
data_complete_sick=pd.concat([patients_pp[0],patients_pp[3]],axis=0)
genes_healthy=list(data_complete_healthy.loc[:, (data_complete_healthy != data_complete_healthy.iloc[0]).any()].columns)
genes_sick=list(data_complete_sick.loc[:, (data_complete_sick != data_complete_sick.iloc[0]).any()].columns)

index_healthy=len(patients_pp[1])
index_sick=len(patients_pp[0])



In [97]:
imputed_global=data_complete_healthy.copy()
imputed_local=sce.pp.magic(ad.AnnData(data_complete_healthy.loc[:,genes_healthy]),knn=80,t='auto',n_pca=175,n_jobs=8,copy=True,verbose=0,solver='approximate')
imputed_global.loc[:,genes_healthy]=imputed_local.X
result=0
sc_imputed1= imputed_global.iloc[index_healthy:,:]
sc_imputed2= imputed_global.iloc[:index_healthy,:]
bulkified1=sc_imputed1.mean(axis=0)
bulkified2=sc_imputed2.mean(axis=0)
bulk1=bulk_train['patient' + str(2)]
bulk2=bulk_train['patient' + str(3)]

result=(spearmanr(bulkified1,bulk1).statistic + spearmanr(bulkified2,bulk2).statistic)/4
result


imputed_global=data_complete_sick.copy()
imputed_local=sce.pp.magic(ad.AnnData(data_complete_sick.loc[:,genes_sick]),knn=80,t='auto',n_pca=175,n_jobs=8,copy=True,verbose=0,solver='approximate')
imputed_global.loc[:,genes_sick]=imputed_local.X
sc_imputed1= imputed_global.iloc[index_sick:,:]
sc_imputed2= imputed_global.iloc[:index_sick,:]
bulkified1=sc_imputed1.mean(axis=0)
bulkified2=sc_imputed2.mean(axis=0)
bulk1=bulk_train['patient' + str(1)]
bulk2=bulk_train['patient' + str(4)]

result+=(spearmanr(bulkified1,bulk1).statistic + spearmanr(bulkified2,bulk2).statistic)/4
result

0.94194410817671

# Last... Using also test data

In [14]:
cells_1=[column for column in sc_train.columns if 'patient1' in column ]
cells_2=[column for column in sc_train.columns if 'patient2' in column ]
cells_3=[column for column in sc_train.columns if 'patient3' in column ]
cells_4=[column for column in sc_train.columns if 'patient4' in column ]
cells_5=[column for column in sc_test.columns if 'patient5' in column ]
cells_6=[column for column in sc_test.columns if 'patient6' in column ]
cells_7=[column for column in sc_test.columns if 'patient7' in column ]


# we transpose the dataframe since we want a cells x genes dataframe for applying MAGIC
sc1_train=sc_train[cells_1].T
sc2_train=sc_train[cells_2].T
sc3_train=sc_train[cells_3].T
sc4_train=sc_train[cells_4].T
sc5_train=sc_test[cells_5].T
sc6_train=sc_test[cells_6].T
sc7_train=sc_test[cells_7].T


patients=[sc1_train,sc2_train,sc3_train,sc4_train,sc5_train,sc6_train,sc7_train]
patients_pp=[]
#after the loop variables are called data1,data2,data3,data4 or are in the patients_pp list
for i,data in enumerate(patients):
    pp_data=pp(data)
    vars().__setitem__('data' + str(i+1), pp_data )
    patients_pp += [pp_data]

In [15]:
data_complete=pd.concat(patients_pp,axis=0)
genes=list(data_complete.loc[:, (data_complete != data_complete.iloc[0]).any()].columns)

lens=[len(x) for x in patients_pp]
indexes=[np.sum(lens[:i+1]) for i in range(4)]

In [22]:
results=[]
k_set ,t_set, n_pca_set= [250], ['auto'] ,[250]
best_params=0
best_score=0
baseline=np.mean([spearmanr(patients_pp[i].mean(axis=0),bulk_train['patient' + str(i+1)]).statistic for i in range(4)])
for k,t,n_pca in product(k_set,t_set,n_pca_set):
    imputed_global=data_complete.copy()
    imputed_local=sce.pp.magic(ad.AnnData(data_complete.loc[:,genes]),knn=k,t=t,n_pca=n_pca,n_jobs=8,copy=True,verbose=0,solver='approximate',random_state=7)
    imputed_global.loc[:,genes]=imputed_local.X
    result=0
    for i,index in enumerate(indexes):
        if i==0: sc_imputed= imputed_global.iloc[:index,:]
        else: sc_imputed= imputed_global.iloc[indexes[i-1]:index,:]
        bulkified=sc_imputed.mean(axis=0)
        bulk=bulk_train['patient' + str(i+1)]
        result+=spearmanr(bulkified,bulk).statistic/4
    if result >= best_score:
        best_params=[k,t,n_pca]
        best_score=result
    results+=[result]
    print(f'parameters: {k,t,n_pca}')
    print(f'Correlation After imputation: {result}\n--------')


print(f'Correlation before imputation: {baseline}')
print(f'BEST CORRELATION AFTER IMPUTATION: {np.max(results)}')
print(f'best parameters: {best_params}')

parameters: (250, 'auto', 250)
Correlation After imputation: 0.9525459800307813
--------
Correlation before imputation: 0.9358073816280966
BEST CORRELATION AFTER IMPUTATION: 0.9525459800307813
best parameters: [250, 'auto', 250]


In [30]:
indexes=[np.sum(lens[:i+1]) for i in range(7)]
indexes


final_result= imputed_global.iloc[indexes[3]:,:]
final_result.to_csv(path_cwd + '/ML4G_Project_2_Data/Final/Imputation.csv')

# Setting up things for Submission

In [24]:
imputation=pd.read_csv('C:/Users/gugli/Documents/GitHub/ml4g_2023/Project_2/ML4G_Project_2_Data/Vollenweider_Michael_Project2/Imputation.csv',index_col=0).T
file_destination='C:/Users/gugli/Documents/GitHub/ml4g_2023/Project_2/ML4G_Project_2_Data/Vollenweider_Michael_Project2/imputed_bulkified.csv'

In [25]:
cells_5=[column for column in imputation.columns if 'patient5' in column ]
cells_6=[column for column in imputation.columns if 'patient6' in column ]
cells_7=[column for column in imputation.columns if 'patient7' in column ]

d={'index':list(imputation.index),'patient5':imputation.loc[:,cells_5].mean(axis=1), 'patient6':imputation.loc[:,cells_6].mean(axis=1), 'patient7':imputation.loc[:,cells_7].mean(axis=1)}
pd.DataFrame(d).reset_index().drop('level_0',axis=1).to_csv(file_destination)